# CTQ Baseline Analysis – Medical Clinic
This notebook computes baseline performance for each Critical-to-Quality (CTQ) metric
using the provided outpatient timestamp dataset.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

file_path = r"/mnt/data/Medical Clinic (1).xlsx"
df = pd.read_excel(file_path)
df.head()

In [ ]:
from datetime import datetime

def combine_dt(date_col, time_col):
    return pd.to_datetime(df[date_col].astype(str) + " " + df[time_col].astype(str))

arrival = combine_dt("Date", "Patient Arrived at the Clinic")
reception = combine_dt("Date", "Patient Spoke with the Reception")
nurse_call = combine_dt("Date", "Patient was Called by the Nurse")
visit_room = combine_dt("Date", "Patient was Guided to the Visit Room")
doctor_meet = combine_dt("Date", "Patient Met the Doctor")
left_clinic = combine_dt("Date", "Patient Left the Clinic")

df["Reception_Time_min"] = (reception - arrival).dt.total_seconds() / 60
df["Nurse_Wait_min"] = (nurse_call - arrival).dt.total_seconds() / 60
df["Doctor_Wait_min"] = (doctor_meet - visit_room).dt.total_seconds() / 60
df["Consult_Duration_min"] = (left_clinic - doctor_meet).dt.total_seconds() / 60

df[["Reception_Time_min", "Nurse_Wait_min", "Doctor_Wait_min", "Consult_Duration_min"]].describe()


In [ ]:
reception_target = 5
nurse_target = 15
doctor_target = 20
consult_min = 5
consult_max = 10

summary = {}

def ctq_stats(series, name, defect_condition):
    s = series.dropna()
    stats = {
        "N": int(s.count()),
        "Mean": s.mean(),
        "Median": s.median(),
        "Std": s.std(),
        "Min": s.min(),
        "Max": s.max(),
    }
    defects = s[defect_condition(s)]
    stats["Defect_Count"] = int(defects.count())
    stats["Defect_Pct"] = defects.count() / s.count() * 100
    summary[name] = stats

ctq_stats(df["Nurse_Wait_min"], "Nurse_Wait_Time", lambda s: s > nurse_target)
ctq_stats(df["Doctor_Wait_min"], "Doctor_Wait_Time", lambda s: s > doctor_target)
ctq_stats(df["Reception_Time_min"], "Reception_Time", lambda s: s > reception_target)
ctq_stats(df["Consult_Duration_min"], "Consult_Duration", lambda s: (s < consult_min) | (s > consult_max))

summary_df = pd.DataFrame(summary).T
summary_df


In [ ]:
ctq_columns = {
    "Reception_Time_min": "Reception Processing Time (min)",
    "Nurse_Wait_min": "Nurse Waiting Time (min)",
    "Doctor_Wait_min": "Doctor Waiting Time (min)",
    "Consult_Duration_min": "Consultation Duration (min)",
}

for col, label in ctq_columns.items():
    plt.figure()
    df[col].hist(bins=10)
    plt.xlabel(label)
    plt.ylabel("Frequency")
    plt.title(f"Histogram of {label}")
    plt.show()


## Interpretation Notes (for report)

- **Nurse Waiting Time (< 15 min target):** All observed values are above the CTQ limit, indicating a 100% defect rate relative to the VOC threshold. This confirms that the nurse call step is the dominant bottleneck in the process.
- **Doctor Waiting Time (< 20 min target):** All observations are within the CTQ target. From a baseline perspective, the doctor step currently meets customer expectations.
- **Reception Time (< 5 min target):** About half of the visits exceed the 5-minute limit, suggesting inconsistent performance at registration and early-stage congestion.
- **Consultation Duration (5–10 min target):** Most consultations are shorter than the minimum CTQ threshold, leading to a high defect rate. This may reflect rushed visits or documentation outside the consultation window.

These baseline results will be used as the starting point for capability analysis and root cause investigation in the Analyze phase.
